## Homework 02: Unsupervised embedding-based MT
*Note: this homework is based on open materials from yandexdataschool [NLP course](https://github.com/yandexdataschool/nlp_course/). Feel free to check this awesome course if you wish to dig deeper.*

*Refined by [Nikolay Karpachev](https://www.linkedin.com/in/nikolay-karpachev-b0146a104/)*

**In this homework** **<font color='red'>YOU</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully), in particular for similar languages, e.g. Ukrainian and Russian. 

### Frament of the Swadesh list for some slavic languages

The Swadesh list is a lexicostatistical stuff. It's named after American linguist Morris Swadesh and contains basic lexis. This list are used to define subgroupings of languages, its relatedness.

So we can see some kind of word invariance for different Slavic languages.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

But the context distribution of these languages demonstrates even more invariance. And we can use this fact for our for our purposes.

## Data

In [1]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

In this notebook we're going to use pretrained word vectors - FastText (original paper - https://arxiv.org/abs/1607.04606).

You can download them from the official [website](https://fasttext.cc/docs/en/crawl-vectors.html). We're going to need embeddings for Russian and Ukrainian languages. Please use word2vec-compatible format (.text).

In [2]:
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")

In [3]:
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")

In [4]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 1.0),
 ('июль', 0.9383153319358826),
 ('сентябрь', 0.9240028858184814),
 ('июнь', 0.9222576022148132),
 ('октябрь', 0.9095539450645447),
 ('ноябрь', 0.8930035829544067),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545796871185303),
 ('февраль', 0.8401416540145874)]

In [5]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 1.0),
 ('липень', 0.9096439480781555),
 ('вересень', 0.9016969203948975),
 ('червень', 0.8992519974708557),
 ('жовтень', 0.8810408115386963),
 ('листопад', 0.8787633776664734),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.8408110737800598),
 ('лютий', 0.8256431221961975)]

In [6]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Stepashka.com', 0.2757962942123413),
 ('ЖИЗНИВадим', 0.25203439593315125),
 ('2Дмитрий', 0.25048112869262695),
 ('2012Дмитрий', 0.24829229712486267),
 ('Ведущий-Алексей', 0.2443869709968567),
 ('Недопустимость', 0.24435287714004517),
 ('2Михаил', 0.23981398344039917),
 ('лексей', 0.23740758001804352),
 ('комплексн', 0.23695147037506104),
 ('персональ', 0.2368222177028656)]

Load small dictionaries for correspoinding words pairs as trainset and testset.

In [34]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r", encoding="utf-8") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [24]:
# !wget -O ukr_rus.train.txt http://tiny.cc/jfgecz

import urllib.request
url = 'http://tiny.cc/jfgecz'
filename = 'ukr_rus.train.txt'
urllib.request.urlretrieve(url, filename)

('ukr_rus.train.txt', <http.client.HTTPMessage at 0x1e2e849e508>)

In [25]:
# !wget -O ukr_rus.test.txt http://tiny.cc/6zoeez
    
url = 'http://tiny.cc/6zoeez'
filename = 'ukr_rus.test.txt'
urllib.request.urlretrieve(url, filename)

('ukr_rus.test.txt', <http.client.HTTPMessage at 0x1e2e84a2708>)

In [35]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")

In [36]:
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

## Embedding space mapping (0.3 pts)

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called Procrustes problem:

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$
or
$$W^*= \arg\min_W ||WX - Y||_F$$

where $||*||_F$ - Frobenius norm.

$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ looks like simple multiple linear regression (without intercept fit). So let's code.

In [39]:
from sklearn.linear_model import LinearRegression

# YOUR CODE HERE

mapping = LinearRegression().fit(X_train, Y_train)

Let's take a look at neigbours of the vector of word _"серпень"_ (_"август"_ in Russian) after linear transform.

In [40]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8531404137611389),
 ('июнь', 0.8402308225631714),
 ('март', 0.8385775089263916),
 ('сентябрь', 0.833186686038971),
 ('февраль', 0.8311493396759033),
 ('октябрь', 0.8278171420097351),
 ('ноябрь', 0.8244150280952454),
 ('июль', 0.8228995203971863),
 ('август', 0.8112362623214722),
 ('январь', 0.80228590965271)]

We can see that neighbourhood of this embedding cosists of different months, but right variant is on the ninth place.

As quality measure we will use precision top-1, top-5 and top-10 (for each transformed Ukrainian embedding we count how many right target pairs are found in top N nearest neighbours in Russian embedding space).

In [46]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        # YOUR CODE HERE
        
        most_similar = ru_emb.most_similar([mapped_vectors[i]], topn=topn)
        most_similar = list(map(lambda x: x[0], most_similar))
        num_matches += int(ru in most_similar)

    precision_val = num_matches / len(pairs)
    return precision_val

In [47]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [48]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [49]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

In [50]:
print(precision_top1)
print(precision_top5)

0.6259541984732825
0.7913486005089059


## Making it better (orthogonal Procrustean problem) (0.3 pts)

It can be shown (see original paper) that a self-consistent linear mapping between semantic spaces should be orthogonal. 
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$W^*= \arg\min_W ||WX - Y||_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$W^*=UV^T$$

In [51]:
import numpy as np

In [52]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    # YOUR CODE GOES HERE
    # compute orthogonal embedding space mapping
    
    u, d, w = np.linalg.svd(X_train.T @ Y_train)
    
    mapping = u @ w

    return mapping

In [53]:
W = learn_transform(X_train, Y_train)

In [54]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('апрель', 0.8245131373405457),
 ('июнь', 0.8056628704071045),
 ('сентябрь', 0.8055761456489563),
 ('март', 0.8032935857772827),
 ('октябрь', 0.7987102270126343),
 ('июль', 0.7946796417236328),
 ('ноябрь', 0.7939636707305908),
 ('август', 0.7938188314437866),
 ('февраль', 0.7923861145973206),
 ('декабрь', 0.7715375423431396)]

In [55]:
print(precision(uk_ru_test, np.matmul(X_test, W)))
print(precision(uk_ru_test, np.matmul(X_test, W), 5))

0.6437659033078881
0.7989821882951654


## Unsupervised embedding-based MT (0.4 pts)

Now, let's build our word embeddings-based translator!

Firstly, download OPUS Tatoeba corpus.

In [59]:
#!wget https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/uk.txt.gz

In [60]:
#!gzip -d ./uk.txt.gz

In [61]:
with open('./uk.txt', 'r', encoding="utf-8") as f:
    uk_corpus = f.readlines()

In [62]:
# To save your time and CPU, feel free to use first 1000 sentences of the corpus
uk_corpus = uk_corpus[:1000]

In [63]:
# Any necessary preprocessing if needed
# YOUR CODE HERE

def tokenizer(token):
    try:
        most_similar = ru_emb.most_similar([np.matmul(uk_emb[token], W)])
        return most_similar[0][0]
    except:
        return token

In [64]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    # YOUR CODE GOES HERE
    
    translated = [tokenizer(x) for x in sentence.split()]
    return " ".join(translated)

In [65]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

Now you can play with your model and try to get as accurate translations as possible. **Note**: one big issue is out-of-vocabulary words. Try to think of various ways of handling it (you can start with translating each of them to a special **UNK** token and then move to more sophisticated approaches). Good luck!

In [70]:
for sent in uk_corpus[::50]:
    print(f'Предложение:\t{sent}Перевод:\t{translate(sent)}\n')

Предложение:	Я вже закінчу коледж, коли ви вернетеся з Америки.
Перевод:	Я уже закончу коледж, когда мы прибежишь со океании

Предложение:	Подивися на той дим.
Перевод:	Посмотри по тот жеребей

Предложение:	У мене є мрія.
Перевод:	Во меня То мрія.

Предложение:	Він чекав на нього до десятої години.
Перевод:	Он ждал по него к десятой часа

Предложение:	Я знаю, що ти багатий.
Перевод:	Я знаю, что ты багатий.

Предложение:	Не твоя справа!
Перевод:	Не моя справа!

Предложение:	Коли Ви народилися?
Перевод:	Когда Вы народилися?

Предложение:	Поверхня повітряної кулі — неевклідовий простір, тому для неї не виконуються правила евклідової геометрії.
Перевод:	поверхность воздушной шары — неевклідовий простір, потому для неё не выполняются правила симметрической геометрії.

Предложение:	Один, два, три, чотири, п'ять, шість, сім, вісім, дев'ять, десять.
Перевод:	Один, два, три, чотири, п'ять, шість, сім, вісім, дев'ять, десять.

Предложение:	Кейт п’є багато молока кожен день.
Перевод:	Рейчел п’є м